SAM(Segmentation Anything Model), Mask R-CNN, DeepLabV3, YOLO-Seg

- 파이프라인
  - 1. 비디오 프레임 추출
    - OpenCV를 이용 동영상을 프레임단위로 나눔
  - 2. 객체 세그먼테이션(SAM사용)
    - SAM을 이용해 각 프레임에서 특정 객체를 픽셀 단위로 분할
  - 3. 마스크를 원본 프레임에 오버레이
    - 세그먼테이션된 마스크를 투명한 색상으로 원본 프레임위에 시각적으로 합성
  - 4. 새로운 동영상
    - 각각 처리한 프레임을 하나의 동영상으로 합침

라이브러리 설치

In [1]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [2]:
!pip install opencv-python numpy matplotlib

In [3]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-p3ua_5wp
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-p3ua_5wp
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=27a94e5caa697750e901d58276f4ec2f726dbefa04d9020c7bae2eaf0f0c5321
  Stored in directory: /tmp/pip-ephem-wheel-cache-y3ma_2d1/wheels/15/d7/bd/05f5f23b7dcbe70cbc6783b06f12143b0cf1a5da5c7b52dcc5
Successfully built segment_anything


In [4]:
!pip install gdown

SAM 모델 가중치 다운로드

In [5]:
import os
import gdown
# 모델 가중치 다운로드
sam_checkpoint = 'sam_vit_h_4b8939.pth'
model_url = 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth'
# 가중치 파일이 없으면 다운로드
if not os.path.exists(sam_checkpoint):
  print('weight downloading...')
  gdown.download(model_url,sam_checkpoint)
else:
  print('weight already exist.')

Downloading...
From: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
To: /content/sam_vit_h_4b8939.pth


weight downloading...


100%|██████████| 2.56G/2.56G [00:21<00:00, 119MB/s]


youtube 동영상 다운로드

In [6]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.0 MB/s eta 0:00:00


In [9]:
# 다운로드할 youtube 영상
youtube_url = 'https://www.youtube.com/shorts/0z7pfc_Q6QQ'
# mp4 형식으로 다운로드
!yt-dlp -f best -o 'sample_vidoe.mp4' {youtube_url}

         To let yt-dlp download and merge the best available formats, simply do not pass any format selection.
         If you know what you are doing and want only the best pre-merged format, use "-f b" instead to suppress this warning
[youtube] Extracting URL: https://www.youtube.com/shorts/0z7pfc_Q6QQ
[youtube] 0z7pfc_Q6QQ: Downloading webpage
[youtube] 0z7pfc_Q6QQ: Downloading tv client config
[youtube] 0z7pfc_Q6QQ: Downloading player f3d47b5a
[youtube] 0z7pfc_Q6QQ: Downloading tv player API JSON
[youtube] 0z7pfc_Q6QQ: Downloading ios player API JSON
[youtube] 0z7pfc_Q6QQ: Downloading m3u8 information
[info] 0z7pfc_Q6QQ: Downloading 1 format(s): 18
[download] Destination: sample_vidoe.mp4
[download] 100% of  645.88KiB in 00:00:00 at 1.66MiB/s


SAM 모델 로드 및 비디오 처리

In [10]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor
from PIL import Image

# sam 모델 로드
model_type = 'vit_h'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device)
predictor = SamPredictor(sam)

# 비디오 파일 로드
video_path = '/content/sample_vidoe.mp4'
cap = cv2.VideoCapture(video_path)
frame_width =  int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = '/content/output_video.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# 프레임별 세그먼테이션 및 시각화
frame_count = 0
while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break  # 동영상 끝
  frame_count += 1
  print(f"Processing frame {frame_count}...")
  # RGB 변환(OpenCV는 RGB를 사용하므로 변환 필요)
  image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  # PIL 이미지로 변환
  image_pil = Image.fromarray(image)
  # sam 모델 입력을 위한 전처리
  predictor.set_image(np.array(image_pil))
  # 관심 객체의 박스 입력(중앙 영역을 임시로 지정)
  h,w,_ = image.shape
  input_box = np.array([w//4, h//4, w*3//4, h*3//4]) # 중앙 사각형 영역
  # sam 예측 수행
  masks, scores, logits = predictor.predict(box = input_box,multimask_output=False)
  mask = masks[0]  # 가장 확률이 높은 마스트 선택
  # 마스크를 원본 이미지에 오버레이
  mask_overlay = np.zeros_like(frame, dtype=np.uint8)
  mask_overlay[mask] = (0,255,0) # 녹색 마스크 적용
  # 투명도 조절
  alpha = 0.5
  blended = cv2.addWeighted(frame, 1-alpha, mask_overlay, alpha, 0)
  # 결과 영상 저장
  out.write(blended)

# 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()
print('segmentatino complete',output_path)

Processing frame 1...
Processing frame 2...
Processing frame 3...
Processing frame 4...
Processing frame 5...
Processing frame 6...
Processing frame 7...
Processing frame 8...
Processing frame 9...
Processing frame 10...
Processing frame 11...
Processing frame 12...
Processing frame 13...
Processing frame 14...
Processing frame 15...
Processing frame 16...
Processing frame 17...
Processing frame 18...
Processing frame 19...
Processing frame 20...
Processing frame 21...
Processing frame 22...
Processing frame 23...
Processing frame 24...
Processing frame 25...
Processing frame 26...
Processing frame 27...
Processing frame 28...
Processing frame 29...
Processing frame 30...
Processing frame 31...
Processing frame 32...
Processing frame 33...
Processing frame 34...
Processing frame 35...
Processing frame 36...
Processing frame 37...
Processing frame 38...
Processing frame 39...
Processing frame 40...
Processing frame 41...
Processing frame 42...
Processing frame 43...
Processing frame 44.

- YOLO + SAM 결합
  - YOLOv8을 사용해서 객체 감지
  - SAM을 활용하여 감지된 객체의 픽셀 단위마스크 생성
  - 세그먼테이션 결과를 비디오에 오버레이하여 저장  

In [11]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python numpy matplotlib
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install ultralytics
!pip install gdown

# sam모델 가중치 VOLOv8 다운로드
# SAM
import os
import gdown
# 모델 가중치 다운로드
sam_checkpoint = 'sam_vit_h_4b8939.pth'
model_url = 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth'
# 가중치 파일이 없으면 다운로드
if not os.path.exists(sam_checkpoint):
  print('weight downloading...')
  gdown.download(model_url,sam_checkpoint)
else:
  print('weight already exist.')

# yolov8
from ultralytics import YOLO
yolo_model = YOLO('yolov8n.pt')

# 다운로드할 youtube 영상
youtube_url = 'https://www.youtube.com/shorts/0z7pfc_Q6QQ'
# mp4 형식으로 다운로드
!yt-dlp -f best -o 'sample_vidoe.mp4' {youtube_url}

# yolo + sam 기반 객체 탐지 및 세그먼테이션


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-o0t19drz
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-o0t19drz
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 23.7 MB/s eta 0:00:00
weight already exist.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 114MB/s]


         To let yt-dlp download and merge the best available formats, simply do not pass any format selection.
         If you know what you are doing and want only the best pre-merged format, use "-f b" instead to suppress this warning
[youtube] Extracting URL: https://www.youtube.com/shorts/0z7pfc_Q6QQ
[youtube] 0z7pfc_Q6QQ: Downloading webpage
[youtube] 0z7pfc_Q6QQ: Downloading tv client config
[youtube] 0z7pfc_Q6QQ: Downloading player f3d47b5a
[youtube] 0z7pfc_Q6QQ: Downloading tv player API JSON
[youtube] 0z7pfc_Q6QQ: Downloading ios player API JSON
[youtube] 0z7pfc_Q6QQ: Downloading m3u8 information
[info] 0z7pfc_Q6QQ: Downloading 1 format(s): 18
[download] sample_vidoe.mp4 has already been downloaded
[download] 100% of  645.88KiB


In [ ]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor
from PIL import Image

# sam 모델 로드
model_type = 'vit_h'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device)
predictor = SamPredictor(sam)

# 비디오 파일 로드
video_path = '/content/sample_vidoe.mp4'
cap = cv2.VideoCapture(video_path)
frame_width =  int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = '/content/output_sam_yolo_video.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# 프레임별 세그먼테이션 및 시각화
frame_count = 0
while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break  # 동영상 끝

  frame_count += 1
  print(f"Processing frame {frame_count}...")

  # yolo 객체 감지
  results = yolo_model(frame)

  for result in results:
    boxes = result.boxes.xyxy.cpu().numpy() # 객체가 감지된 박스(x1, y1, x2, y2)

    for box in boxes:
      x1, y1, x2, y2 = map(int,box)

      # RGB 변환(OpenCV는 RGB를 사용하므로 변환 필요)
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      predictor.set_image(np.array(image))

      # yolo감지 박스를 sam 입력으로 사용
      input_box = np.array([x1, y1, x2, y2])

      # sam 예측 수행
      masks, scores, logits = predictor.predict(box = input_box,multimask_output=False)
      mask = masks[0]  # 가장 확률이 높은 마스트 선택

      # 마스크를 원본 이미지에 오버레이
      mask_overlay = np.zeros_like(frame, dtype=np.uint8)
      mask_overlay[mask] = (0,255,0) # 녹색 마스크 적용

      # 투명도 조절
      alpha = 0.5
      blended = cv2.addWeighted(frame, 1-alpha, mask_overlay, alpha, 0)

      # 감지된 객체 박스 그리기
      cv2.rectangle(blended,(x1,y1), (x2,y2), (0,0,255),2)

      # 객체 ID 출력(yolo가 클래스제공)
      label = result.names[result.boxes.cls[0].item()]
      cv2.putText(blended, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

  # 결과 영상 저장
  out.write(blended)

# 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()
print('yolo+segmentatino complete',output_path)

Processing frame 1...

0: 640x384 13 persons, 1 suitcase, 10.7ms
Speed: 2.2ms preprocess, 10.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)
Processing frame 2...

0: 640x384 12 persons, 1 suitcase, 9.5ms
Speed: 1.5ms preprocess, 9.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)
